# neor - experiments in smart objects to front for neo4j

* https://docs.python.org/3/howto/descriptor.html#id22

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neo4j
import tools.neotools as nj
import grexutils as gu
from grex_tasks import add, run_epp_code_strings

/home/jovyan/work/nbs/grex_tasks.py:11: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


In [3]:
driver = neo4j.GraphDatabase.driver("neo4j://neo4j:7687", auth=("neo4j", "test"))
driver.verify_connectivity()

<ipython-input-3-e75ba85668dd>:2: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


{IPv4Address(('neo4j', 7687)): [{'ttl': 300,
   'servers': [{'addresses': ['neo4j:7687'], 'role': 'WRITE'},
    {'addresses': ['neo4j:7687'], 'role': 'READ'},
    {'addresses': ['neo4j:7687'], 'role': 'ROUTE'}]}]}

In [4]:
ex_key = gu.get_experiment_key_from_name(driver, name='t3')
ex_key

'7R42cwhPZvhpmEUd51W5xQ'

In [5]:
class UnikeyedObjectProperty:
    def __set_name__(self, owner, attr_name):
        #print(owner.__name__)
        self.owner = owner
        self.attr_name = attr_name
        self.fetch_q ="MATCH (n:%s {unikey: $unikey}) RETURN n.%s as %s" % (owner.__name__, attr_name, attr_name)
        self.store_q = ""
        
    def __get__(self, obj, objtype=None):
        return nj.query_read_return_list(obj.driver, self.fetch_q, unikey=obj.unikey)[0][self.attr_name]
    
    def __set__(self, obj, value):
        return

In [6]:
class Experiment:
    name = UnikeyedObjectProperty()
    ts = UnikeyedObjectProperty()
    
    def __init__(self, driver, unikey):
        self.driver = driver
        self.unikey = unikey

In [7]:
class Procedure:
    name = UnikeyedObjectProperty()
    ts = UnikeyedObjectProperty()
    code_strings = UnikeyedObjectProperty()
    
    def __init__(self, driver, unikey):
        self.driver = driver
        self.unikey = unikey

In [29]:
class Parameters:
    name = UnikeyedObjectProperty()
    ts = UnikeyedObjectProperty()
    prepend_code_strings = UnikeyedObjectProperty()
    append_code_strings = UnikeyedObjectProperty()
    trial = UnikeyedObjectProperty()
    
    def __init__(self, driver, unikey):
        self.driver = driver
        self.unikey = unikey

In [53]:
class Net:
    batches_from_start = UnikeyedObjectProperty()
    head = UnikeyedObjectProperty()
    ksv = UnikeyedObjectProperty()
    loss = UnikeyedObjectProperty()
    shorthand = UnikeyedObjectProperty()
    ts = UnikeyedObjectProperty()
    
    def __init__(self, driver, unikey):
        self.driver = driver
        self.unikey = unikey

In [54]:
net_key = 'bkZSa4I-1wvU6xDlexzghQ'
net = Net(driver, net_key)

In [55]:
net.ksv, net.shorthand, net.loss, net.ts, net.head

('46901af5cc1e8911', '1x8tx8tx3tx3t', 2.4346133939738834, 1615984725090, True)

In [30]:
proc_key = 'RAVTYeYlXdjVx_1SUlrwHg'

In [31]:
proc = Procedure(driver, proc_key)

In [32]:
proc.name, proc.ts

('yet more Train ADCs', 1615861288364)

In [33]:
ex = Experiment(driver, '7R42cwhPZvhpmEUd51W5xQ')

In [34]:
ex.name, ex.ts

('t3', 1615778080564)

In [35]:
ex.unikey

'7R42cwhPZvhpmEUd51W5xQ'

In [36]:
ex.driver

In [37]:
param_key = 'nHJKaTreNkRyE5L4HhUFUQ'

In [38]:
param = Parameters(driver, param_key)

In [39]:
param.name, param.ts, param.trial, param.prepend_code_strings, param.append_code_strings

('go crazy 2',
 1615861288920,
 2,
 ['random_seed = 1853412476',
  "trial_number = '2'",
  "parameters_unikey = 'nHJKaTreNkRyE5L4HhUFUQ'"],
 ['net.eps = 5.0'])

In [40]:
net_key = 'tSa4SW7CYxhFnwLu8_rchg'

In [43]:
node_type_str = 'net'
name = ''
q = """
MATCH (n:%s {unikey: $key})
RETURN n
""" % (node_type_str,)
nj.query_read_return_list(driver, q, key=net_key)[0]['n'].keys()

dict_keys(['loss', 'batches_from_start', 'unikey', 'shorthand', 'ts', 'ksv'])